<a href="https://colab.research.google.com/github/JakeOh/202011_itw_bd21/blob/main/lab_ml/ml20_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN(Recurrent Neural Netowrk, 순환 신경망)

시계열 데이터, 텍스트 데이터와 같이 순서가 중요한 데이터(순차 데이터, sequence)를 다룰 때 많이 사용하는 신경망.

# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

# 단어 수준 one-hot encoding

## 단어-인덱스 매핑

In [2]:
sentence = 'The cat sat on the table.'

In [5]:
word_index = {}  # 단어를 키로 하고, 숫자를 값(value)로 저장할 dict
for word in sentence.split():  # 하나의 문장 공백으로 구분된 단어 리스트에서 반복하면서
    if word not in word_index:  # 단어가 dict에 없다면,
        word_index[word] = len(word_index) + 1
print(word_index)

{'The': 1, 'cat': 2, 'sat': 3, 'on': 4, 'the': 5, 'table.': 6}


In [6]:
samples = ['The cat sat on the table.', 'The dog ate my laptop.']

In [7]:
# samples 리스트에 등장하는 단어들의 word-index 사전을 만드세요.
word_index = {}
for s in samples:
    for w in s.split():
        if w not in word_index:
            word_index[w] = len(word_index) + 1
print(word_index)

{'The': 1, 'cat': 2, 'sat': 3, 'on': 4, 'the': 5, 'table.': 6, 'dog': 7, 'ate': 8, 'my': 9, 'laptop.': 10}


## 텍스트를 숫자들의 리스트로 변환

In [8]:
samples_to_sequences = []  # 숫자들의 리스트를 저장할 리스트
for s in samples:
    sequence = []  # 하나의 문장을 숫자들로 변환한 리스트
    for w in s.split():
        sequence.append(word_index[w])  # 사전에서 단어에 매핑된 인덱스를 찾아서 리스트에 추가.
    samples_to_sequences.append(sequence)

print(samples_to_sequences)
#> [[1, 2, 3, 4, 5, 6], [1, 7, 8, 9, 10]]

[[1, 2, 3, 4, 5, 6], [1, 7, 8, 9, 10]]


샘플 문장들마다 사용된 단어의 개수가 다르기 때문에 scikit-learn, keras 패키지의 모델들을 훈련시킬 수 없다.

## One-hot encoding

In [12]:
max_rows = 10  # 한 문장의 최대 단어 수
max_cols = max(word_index.values()) + 1  # word_index의 값 = column index
samples_1hot = np.zeros(shape=(len(samples), max_rows, max_cols))
for i, s in enumerate(samples):
    for j, w in list(enumerate(s.split()))[:max_rows]:
        index = word_index[w]
        samples_1hot[i, j, index] = 1

print(samples_1hot)

[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
